# NEMU：ISA 参考

我们提出了一个类似于 Spike 的指令集模拟器 NEMU。通过优化技术，NEMU 实现了与 QEMU 类似的性能。此外，NEMU 还提供了一组 API 来帮助 XiangShan 比较和验证架构状态。

NEMU 提供两组默认配置：
- xxx_defconfig：xxx 目标的独立运行模式的默认配置
- xxx-ref_defconfig：xxx 作为 difftest 协同仿真模式的默认配置

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null

cd $NEMU_HOME
### If you are building NEMU for the first time,
### you will need to run the following command to configure it.
### Here we have configured it in advance.
# make menuconfig
make clean
make riscv64-xs_defconfig
make -j

make clean-softfloat
make riscv64-xs-ref_defconfig
make -j

在 NEMU 上运行 Coremark 工作负载。

In [ ]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NEMU_HOME

# 以批模式运行 workload
./build/riscv64-nemu-interpreter \
    -b \
    ${READY2RUN_HOME}/hello-riscv64-xs.bin

# Difftest：ISA 协同仿真框架

我们提出了一个 ISA 协同仿真框架 Difftest。其基本流程如下：一旦我们的 RTL 处理器提交指令/更新其他状态，ISA 模拟器就会执行相同的指令。Difftest 会比较 DUT 和 REF 之间的架构状态。如果存在差异，它将停止运行并报告错误。否则，它将继续运行。

在 XiangShan 上运行工作负载，并使用 NEMU 进行差分测试。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

${READY2RUN_HOME}/emu \
    -i ${READY2RUN_HOME}/hello-riscv64-xs.bin \
    --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so \
    2>/dev/null

可以在预编译好的，注入 bug 的香山处理器上运行工作负载，并使用 NEMU 进行差异测试。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

${READY2RUN_HOME}/emu-alu-err \
    -i ${READY2RUN_HOME}/hello-riscv64-xs.bin \
    --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so \
    2>/dev/null || true # tutorial：添加 "|| true" 避免 notebook 报错，实际使用不需要

可以看到执行 pc = 0x80000dce 处的指令时，仿真和模拟器执行产生了不一样的结果，仿真的体系结构寄存器 a6 值为 0，而模拟器为 0x800040b6。

在 Difftest 检测到错误后，我们可以再次运行仿真，这次在 difftest 报告的出错周期数附近启用波形输出。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd $NOOP_HOME
rm -f ./build/*.vcd

${READY2RUN_HOME}/emu-alu-err \
    -i ${READY2RUN_HOME}/hello-riscv64-xs.bin \
    --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so \
    -b 8000 \
    -e 10000 \
    --dump-wave \
    2>/dev/null || true

echo -n "Dump wave: "
realpath ./build/*.vcd